In [ ]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim
import torch.nn.utils.rnn as rnn

In [ ]:
with open("../../corpus/NTT/persona.json", "r", encoding="utf-8") as f:
    convs = json.load(f)

In [ ]:
import random
random.seed(0)

def make_Xy(convs, n=4, rate=3):

    X_str = []
    y = []

    all_utt = []
    for did in tqdm( convs["convs"] ) :
        dids = list( did.keys() )[0]
        all_utt += did[dids]
    random.shuffle(all_utt)

    j = 0

    for did in tqdm( convs["convs"] ):
        dids = list( did.keys() )[0]
        conv = did[dids]
        # print(conv)
        for i in range(n-1, len(conv)):
            p = (i-n+1)
            # print(i, "[{0}:{1}]".format(p, p+n), conv[p:p+n-1])
            # 正例
            if i%rate != 0:
                X_str.append( conv[p:p+n] )
                y.append(0)
                # print(i, conv[p:p+n])
            # 負例
            else:
                X_str.append( conv[p:p+n-1]+[all_utt[j]] )
                j += 1
                y.append(1)
    
    return X_str, y

In [ ]:
# n: 発話長， rate: エラー発話の確率
X_str_topic, y = make_Xy(convs, n=4)

In [ ]:
X_str_topic_ = X_str_topic[::30]
y_ = y[::30]